E1
1. fcos czy EfficientNet- ktory sie szybciej trenuje
2. ustalenie miary jakosci (uwzględniającą class imbalance)
3. opracowac architekture aplikacji -> najpierw lokalizować dokładną pozycję samochodu, a dopiero później na przyciętym obrazie robić klasyfikację marka-model
   1. Ze względu na proponowane do zaimplementowania architektury **lepszym rozwiązaniem będzie używanie zbioru z "boxami" i użycie połaczonych do uczenia modelu klasyfikatora** ew. użycie 2 zbioru do douczania z transferem. 
4. zaimplementowac wybrany model
5. opisac to slownomuzycznie

E2
1. scrapper / zrobienie zdjec samochodow
2. transfer / dopasowanie modelu do zmiennej wielkosci zdjec (zapewnić możliwość rozszerzenia klas)
3. 

E3 (optional)
App

In [15]:
import pandas as pd
from pathlib import Path
import json

In [8]:
url_data = 'http://imagenet.stanford.edu/internal/car196/cars_train.tgz'
train_file = 'data/stanford/cars_train.tgz'

data_main = Path('./data/')
output_data = data_main / 'stanford' / 

In [10]:
labels = pd.read_csv(output_data/'devkit/labels_df.csv')
labels.head()
labels.tail()

,filename,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class_id,class_name,is_test,filename_cropped,bbox_h,bbox_w
16180,test_08037.jpg,49,57,1169,669,63,Chevrolet Sonic Sedan 2012,1,test_08037.jpg,613,1121
16181,test_08038.jpg,23,18,640,459,16,Audi V8 Sedan 1994,1,test_08038.jpg,442,618
16182,test_08039.jpg,33,27,602,252,17,Audi 100 Sedan 1994,1,test_08039.jpg,226,570
16183,test_08040.jpg,33,142,521,376,38,BMW Z4 Convertible 2012,1,test_08040.jpg,235,489
16184,test_08041.jpg,77,73,506,380,32,BMW X5 SUV 2007,1,test_08041.jpg,308,430


In [13]:
def build_annotation_json_from_frame(frame):
  images = []
  annotations = []
  for _, row in frame.head(2).iterrows():
    im_id = row.filename.split('.')[0]
    images.append({
        "file_name": row.filename,
        "id": im_id,
    })
    annotations.append({
        'image_id':im_id,
        'id':im_id,
        'bbox':[row.bbox_x1, row.bbox_y1, row.bbox_x2, row.bbox_y2],
        'category_id': row.class_id,
    })
  categories = list(frame[['class_id', 'class_name']]
                .groupby(['class_id', 'class_name'])
                .size()
                .reset_index()
                .drop(0, axis=1)
                .rename({'class_id': 'id'}, axis=1)
                .to_dict('index')
                .values())
  return {
      'images': images,
      'annotations': annotations,
      'categories': categories
  }

In [16]:
annotations_json_file = output_data / "annotations.json"
with open(annotations_json_file, "w") as p: 
     p.write(json.dumps(build_annotation_json_from_frame(labels)))

In [18]:
cfg = Config.fromfile('mmdetection/configs/fcos/fcos_center_r50_caffe_fpn_gn_1x_4gpu.py.py')

cfg['data']['imgs_per_gpu'] = 64
cfg['data']['workers_per_gpu'] = 1

NameError: name 'Config' is not defined

In [17]:
data_root = output_data
dataset_type = 'CocoDataset'
cfg.data_root = data_root
img_norm_cfg = dict(
    mean=[102.9801, 115.9465, 122.7717], std=[1.0, 1.0, 1.0], to_rgb=False)


train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

data = dict(
    imgs_per_gpu=1,
    workers_per_gpu=1,
    train=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=train_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=test_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=test_pipeline))

cfg['data'] = data

NameError: name 'cfg' is not defined